In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from data_integration import *

/home/jacobgdt/anaconda3/envs/cling/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [82]:
twtc = load_twtc()

stats = load_stats_df(twtc)
stats = stats.drop(['fg_season_id', 'am_season_id'], axis=1)

df = pd.concat([twtc, stats], axis=1)

df = fill_missing_data(df, stats.columns)
df = df.replace('*.**', np.nan).dropna(subset=stats.columns)

print(df.shape)
print(f'Non-empty rows: {(df[stats.columns].sum(axis=1) != 0).sum()}')
df.head()

(9214, 65)
Non-empty rows: 7969


,name,key_mlbam,key_fangraphs,key_bbref,key_bbref_minors,key_uuid,mlb_played_first,birthdate,debut_age,age,...,SB,Hld,Pitches,BB_pit,OPS,AB,SV,BB,W,SLG
0,**Luke Heimlich,-1,18360,NaN,NaN,NaN,NaN,NaN,NaN,22.3,...,1.0,0.0,1243.0,0.0,0.809597,291.0,0.0,26.0,0.0,0.463918
1,**Noah Song,679977,sa917943,NaN,NaN,NaN,NaN,NaN,NaN,22.0,...,17.0,0.0,1436.0,0.0,0.685559,318.0,0.0,54.0,0.0,0.336478
2,A.J. Alexy,669935,NaN,NaN,alexy-000aj-,c5bd95ff-593c-4875-b48e-012da0caca64,NaN,1998-04-21T00:00:00.000Z,NaN,19.7,...,0.0,0.0,0.0,3.0,0.000000,0.0,0.0,0.0,1.0,0.000000
3,A.J. Alexy,669935,NaN,NaN,alexy-000aj-,c5bd95ff-593c-4875-b48e-012da0caca64,NaN,1998-04-21T00:00:00.000Z,NaN,18.7,...,0.0,0.0,0.0,3.0,0.000000,0.0,0.0,0.0,1.0,0.000000
4,A.J. Cole,595918,11467,coleaj01,cole--001aj-,0596c185-ae62-436a-92b9-79b9a145d64d,2015.0,1992-01-05T00:00:00.000Z,23.0,22.0,...,0.0,0.0,32.0,15.0,0.000000,8.0,0.0,0.0,6.0,0.000000


In [72]:
stats.columns

Index(['OBP', 'CS', 'WHIP', 'HR', 'ERA', 'SO', 'H', 'SO_pit', 'L', 'R',
       'Level', 'RBI', 'Age', 'Age_pit', '3B', 'BS', 'GP', 'TBF', '2B', 'AVG',
       'HR_pit', 'IP', 'SB', 'Hld', 'Pitches', 'BB_pit', 'OPS', 'AB', 'SV',
       'BB', 'W', 'SLG'],
      dtype='object')

In [74]:
lookup = lambda col: df.loc[~df[col].astype(str).apply(lambda x: x.replace('.', '', 1).isdigit()), ['clean_name', col]]

In [76]:
[(c, df[c].dtype) for c in stats.columns]

[('OBP', dtype('float64')),
 ('CS', dtype('float64')),
 ('WHIP', dtype('float64')),
 ('HR', dtype('float64')),
 ('ERA', dtype('float64')),
 ('SO', dtype('float64')),
 ('H', dtype('float64')),
 ('SO_pit', dtype('float64')),
 ('L', dtype('float64')),
 ('R', dtype('float64')),
 ('Level', dtype('O')),
 ('RBI', dtype('float64')),
 ('Age', dtype('O')),
 ('Age_pit', dtype('O')),
 ('3B', dtype('float64')),
 ('BS', dtype('float64')),
 ('GP', dtype('float64')),
 ('TBF', dtype('float64')),
 ('2B', dtype('float64')),
 ('AVG', dtype('float64')),
 ('HR_pit', dtype('float64')),
 ('IP', dtype('float64')),
 ('SB', dtype('float64')),
 ('Hld', dtype('float64')),
 ('Pitches', dtype('float64')),
 ('BB_pit', dtype('float64')),
 ('OPS', dtype('float64')),
 ('AB', dtype('float64')),
 ('SV', dtype('float64')),
 ('BB', dtype('float64')),
 ('W', dtype('float64')),
 ('SLG', dtype('float64'))]

In [83]:
# Otani has MLB stats
df[(df[stats.columns].sum(axis=1) != 0) & (df.Level != 'MLB')].to_csv('joined_stats.csv', index=False)